In [6]:
import pandas as pd
import numpy as np
import glob

## Demand analysis

In [7]:
# Create the location id to filter the trip that pick up and dropoff only in Manhatton island 
location_ids = [
    4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116,
    120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163,
    164, 166, 170, 186, 194, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239,
    243, 244, 246, 249, 261, 262, 263
]

# Make a file pattern to match the CSV files
file_pattern = r"C:\Users\yanzh\Desktop\Code\archive\2019\yellow_tripdata_2019-*.csv"

# Get a list of all CSV files in a directory

# Initialize an empty dictionary to store filtered DataFrames
dfs = {}

csv_files = glob.glob(file_pattern)

# Loop through the list of file paths
for i, file_path in enumerate(csv_files, start=1):
    # Read the CSV file into a Dataframe
    Before_filter_df = pd.read_csv(file_path)

    # Filter the Dataframe
    df = Before_filter_df[Before_filter_df['PULocationID'].isin(location_ids) & Before_filter_df['DOLocationID'].isin(location_ids)]

    # Drop the data from useless columns
    df = df.drop(columns=['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'])

    # Delete the data that travel distance = 0, beacasue we don't consider the situation where trips in the same zone. This data still need to be reconsidered.
    df = df[df['PULocationID'] != df['DOLocationID']]
    
    # Delete the data that fare amount is negative.
    df = df[df['fare_amount'] >= 0]

    # Store the filtered DataFrame in the dictionary
    dfs[f'df{i}'] = df

    '''
    dfs is the dictonary that used to store each filtered Dataframe. The use of a dictionary allows us to assoicate each Dataframe with a unique key
    that can retrieve the Dataframe later on.

    f'df{i}' can be regarded as a key. 

    So 'df1', 'df2' are the keys in the dictionary, each assoicated with its repective filtered DataFrame

    '''

C:\Users\yanzh\AppData\Local\Temp\ipykernel_18012\2821083548.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  Before_filter_df = pd.read_csv(file_path)
C:\Users\yanzh\AppData\Local\Temp\ipykernel_18012\2821083548.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  Before_filter_df = pd.read_csv(file_path)
C:\Users\yanzh\AppData\Local\Temp\ipykernel_18012\2821083548.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  Before_filter_df = pd.read_csv(file_path)
C:\Users\yanzh\AppData\Local\Temp\ipykernel_18012\2821083548.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  Before_filter_df = pd.read_csv(file_path)
C:\Users\yanzh\AppData\Local\Temp\ipykernel_18012\2821083548.py:22: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set lo

In [145]:
df = dfs[f'df{6}']
df['starting_time'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['ending_time'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration_time'] = (df['ending_time'] - df['starting_time']).dt.total_seconds() / 60
df.rename(columns={'PULocationID': 'origin_ID'}, inplace=True)
df.rename(columns={'DOLocationID': 'destination_ID'}, inplace=True)
df = df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'payment_type'])
df
df

,trip_distance,origin_ID,destination_ID,fare_amount,total_amount,starting_time,ending_time,duration_time
1,0.00,262,263,2.5,6.30,2019-06-01 00:06:31,2019-06-01 00:06:52,0.350000
4,1.70,113,148,9.5,15.95,2019-06-01 00:03:25,2019-06-01 00:15:42,12.283333
5,1.60,79,125,9.5,14.30,2019-06-01 00:28:31,2019-06-01 00:39:23,10.866667
6,0.60,211,148,4.5,8.30,2019-06-01 00:46:46,2019-06-01 00:50:55,4.150000
7,1.20,79,249,7.5,12.30,2019-06-01 00:54:49,2019-06-01 01:02:57,8.133333
...,...,...,...,...,...,...,...,...
6941019,0.90,68,158,11.0,16.80,2019-06-30 23:23:03,2019-06-30 23:39:48,16.750000
6941020,0.50,246,90,6.0,9.80,2019-06-30 23:50:22,2019-06-30 23:57:01,6.650000
6941021,0.20,90,186,3.5,8.75,2019-06-30 23:58:32,2019-07-01 00:00:42,2.166667
6941022,1.38,140,163,7.5,13.56,2019-06-30 23:23:10,2019-06-30 23:30:45,7.583333


In [146]:
df['day_type'] = df['starting_time'].dt.dayofweek.apply(lambda x: 'weekend' if x >=5 else 'weekday')
df['hour'] = df['starting_time'].apply(lambda x: f'{x.hour}')

In [147]:
df['hour'] = df['hour'].astype(int)

def time_category(row):
    if row['day_type'] == 'weekend':
        return'weekend'
    elif row['day_type'] == 'weekday' and (8 <= row['hour'] < 10 or 17 <= row['hour'] < 19):
        return 'weekday peakhour'
    else:
        return 'weekday offhour'

df['time_category'] = df.apply(time_category, axis=1)

In [148]:
df

,trip_distance,origin_ID,destination_ID,fare_amount,total_amount,starting_time,ending_time,duration_time,day_type,hour,time_category
1,0.00,262,263,2.5,6.30,2019-06-01 00:06:31,2019-06-01 00:06:52,0.350000,weekend,0,weekend
4,1.70,113,148,9.5,15.95,2019-06-01 00:03:25,2019-06-01 00:15:42,12.283333,weekend,0,weekend
5,1.60,79,125,9.5,14.30,2019-06-01 00:28:31,2019-06-01 00:39:23,10.866667,weekend,0,weekend
6,0.60,211,148,4.5,8.30,2019-06-01 00:46:46,2019-06-01 00:50:55,4.150000,weekend,0,weekend
7,1.20,79,249,7.5,12.30,2019-06-01 00:54:49,2019-06-01 01:02:57,8.133333,weekend,0,weekend
...,...,...,...,...,...,...,...,...,...,...,...
6941019,0.90,68,158,11.0,16.80,2019-06-30 23:23:03,2019-06-30 23:39:48,16.750000,weekend,23,weekend
6941020,0.50,246,90,6.0,9.80,2019-06-30 23:50:22,2019-06-30 23:57:01,6.650000,weekend,23,weekend
6941021,0.20,90,186,3.5,8.75,2019-06-30 23:58:32,2019-07-01 00:00:42,2.166667,weekend,23,weekend
6941022,1.38,140,163,7.5,13.56,2019-06-30 23:23:10,2019-06-30 23:30:45,7.583333,weekend,23,weekend


In [156]:
demand_df = df.groupby(['origin_ID', 'destination_ID', 'hour']).size().reset_index(name='count')

In [157]:
demand_df['count'].describe()

count    79190.000000
mean        68.927794
std        134.480736
min          1.000000
25%          5.000000
50%         20.000000
75%         72.000000
max       3586.000000
Name: count, dtype: float64

So here we divide demand level into three levels: low demand ( lower than 5 trips), medium demand(6 trips to 70 trips), and high demand (larger than 70 trips)

In [175]:
df.groupby(['origin_ID', 'destination_ID']).size().reset_index(name='count')

,origin_ID,destination_ID,count
0,4,12,3
1,4,13,124
2,4,24,11
3,4,41,34
4,4,42,30
...,...,...,...
3934,263,244,310
3935,263,246,604
3936,263,249,328
3937,263,261,129


## Scenarios Definition

Now we can have 24 * 3939 * 3 = 283608 scenarios, and **subsequent reductions** may occur. 

In [169]:
def demand_category(x):
    if x < 6:
        return 'low demand'
    elif 6 <= x <= 70:
        return 'medium demand'
    else:
        return 'high demand'
        

demand_df['demand_level'] = demand_df['count'].apply(demand_category)
demand_df = demand_df.drop(['count'], axis=1)
demand_df

,origin_ID,destination_ID,hour,demand_level
0,4,12,13,low demand
1,4,12,19,low demand
2,4,12,20,low demand
3,4,13,0,low demand
4,4,13,1,medium demand
...,...,...,...,...
79185,263,262,19,high demand
79186,263,262,20,high demand
79187,263,262,21,high demand
79188,263,262,22,high demand


In [180]:
existing_trips_df = df.groupby(['origin_ID', 'destination_ID']).size().reset_index(name='count')
existing_trips_df

,origin_ID,destination_ID,count
0,4,12,3
1,4,13,124
2,4,24,11
3,4,41,34
4,4,42,30
...,...,...,...
3934,263,244,310
3935,263,246,604
3936,263,249,328
3937,263,261,129


In [186]:
# Create a new list of scenario
scenarios = []

for hour in range(24):
    for index, row in existing_trips_df.iterrows():
        for level in range(3):
            scenarios.append((hour, row['origin_ID'], row['destination_ID'], level))

In [189]:
scenarios.to_csv('scenarios', index=False)

AttributeError: 'list' object has no attribute 'to_csv'